In [24]:
# This script compares embedded radio transcripts and fact-checks,
# returning a CSV file of potential misinformation

# ensure libretranslate is running on port 5000 before running this script

import pandas as pd
import os
import numpy as np
from pathlib import Path
from openai import OpenAI
from libretranslatepy import LibreTranslateAPI


In [25]:

number_return_values = 1

# add your OpenAI API key
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-proj-HSrHGyOHtEDtr9loLzRjT3BlbkFJTUFaqCLYDEs3B7qpBz7z"))

embedded_transcripts_path= Path().cwd().parent.joinpath('data/embedded_transcripts/embedded_transcripts.csv')


In [26]:

print('Importing embedded transcripts...')

transcript_df = pd.read_csv(embedded_transcripts_path)

Importing embedded transcripts...


In [ ]:
# there is something wrong with at least one embedding, and I need to figure out how to either skip it or make it work
# it may be from assigning errors as NaN in the other script; may have to change that to 0 or something
for embedding in transcript_df['transcription_embedding']:
    try embedding.

In [42]:
transcript_df['transcription_embedding'] = transcript_df['transcription_embedding'].apply(eval).apply(np.array)

TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
transcript_df['transcription_embedding'] = transcript_df['transcription_embedding'].apply(eval).apply(np.array)

In [27]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [28]:
print("Importing false statements with embeddings...")

embedded_false_statements_path = Path().cwd().parent.joinpath('data/factchecked_statements/embedded_false_statements.csv')

false_text_df = pd.read_csv(embedded_false_statements_path)

false_text_df['statement_embedding'] = false_text_df['statement_embedding'].apply(eval).apply(np.array)

false_text_df['statement'] = false_text_df['statement'].replace('\'', '').replace('\"', '').replace('\"', '').replace('.','').replace('?','').replace('!','')


Importing false statements with embeddings...


In [29]:

def cosine_similarity(vec1, vec2):
    # Normalize each vector to unit length
    vec1_norm = vec1 / np.linalg.norm(vec1)
    vec2_norm = vec2 / np.linalg.norm(vec2)
  
    # Calculate dot product between normalized vectors
    similarity = np.dot(vec1_norm, vec2_norm)
    return similarity


In [30]:

# if the transcript segment is shorter than 150 characters, this will use the context segment instead
def search_false_statements(search_terms,false_text_df=false_text_df, n=1):
    transcription_embedding = search_terms.transcription_embedding
    context_embedding = search_terms.transcription_with_context_embedding
    if len(transcription_embedding) > 150:
        false_text_df['similarities'] = false_text_df['statement_embedding'].apply(lambda x: cosine_similarity(x, transcription_embedding))
    else:
        false_text_df['similarities'] = false_text_df['statement_embedding'].apply(lambda x: cosine_similarity(x, context_embedding))
    results = (
        false_text_df.sort_values("similarities", ascending=False).head(n)
    )
    return results

In [31]:
# if the transcript segment is shorter than 200 characters, this will use the context segment instead
def search_false_statements(search_terms,false_text_df=false_text_df, n=1):
    transcription_embedding = search_terms.transcription_embedding
    context_embedding = search_terms.transcription_with_context_embedding
    if len(transcription_embedding) > 200:
        false_text_df['similarities'] = false_text_df['statement_embedding'].apply(lambda x: cosine_similarity(x, transcription_embedding))
    else:
        false_text_df['similarities'] = false_text_df['statement_embedding'].apply(lambda x: cosine_similarity(x, context_embedding))
    results = (
        false_text_df.sort_values("similarities", ascending=False).head(n)
    )
    return results

In [40]:
top_matches_json = []

for row in transcript_df.itertuples(name='segment'):
    match_index = row.Index
    results = search_false_statements(row)
    top_match = results.statement.values
    similarity = results.similarities
    top_matches = {}
    top_matches["index"] = f"{match_index}"
    top_matches["filename"] = row.file_name
    top_matches["input_statement"] = row.transcription
    top_matches["checked_false_statement"] = f"{top_match}"
    top_matches["similarity"] = f"{similarity}"
    top_matches_json.append(top_matches)

ValueError: could not convert string to float: '[0.007744003087282181, -0.036393869668245316, -0.007606593891978264, 0.02357553318142891, -0.012307959608733654, -0.01184665597975254, 0.032958634197711945, -0.04255766421556473, -0.021848099306225777, 0.007297422271221876, 0.007150197867304087, 0.06371872127056122, -0.023222194984555244, -0.04785774275660515, 0.013623164035379887, 0.03908316791057587, -0.055866751819849014, 0.009201524779200554, 0.036845359951257706, 0.024596288800239563, 0.00598712544888258, 0.01199388038367033, 0.013073526322841644, 0.02589186280965805, 0.026029273867607117, -0.03219306841492653, 0.02555815503001213, 0.010953495278954506, 0.028718572109937668, 0.004127190448343754, 0.02141624130308628, -0.008043359033763409, -0.005368783138692379, -0.013603533618152142, -0.02332034334540367, -0.009971998631954193, -0.019502324983477592, -0.026500390842556953, 0.020316965878009796, 0.0006149685359559953, 0.04047689214348793, 0.02917006053030491, 0.02477295882999897, -0.

In [35]:

top_matches_json = []

def search_all_transcripts():
    errors = 0
    for row in transcript_df.itertuples(name='segment'):
        match_index = row.Index
        try:
            results = search_false_statements(row)
            top_match = results.statement.values
            similarity = results.similarities
            top_matches = {}
            top_matches["index"] = f"{match_index}"
            top_matches["filename"] = row.file_name
            top_matches["input_statement"] = row.transcription
            top_matches["checked_false_statement"] = f"{top_match}"
            top_matches["similarity"] = f"{similarity}"
            top_matches_json.append(top_matches)
            print_text = f"Finished checking {match_index + 1} of {len(transcript_df)} statements -- {int((match_index + 1)/len(exploded_transcript_df)*100)}% complete         "
            print("\r", print_text, end="")
        except:
            errors += 1
            print(f"Error checking statement {match_index + 1}")
    print(f"Finished checking all statements with {errors} error(s)")

print("Starting search of all transcripts...")

search_all_transcripts()

print("Creating DataFrame...")

top_matches_df = pd.DataFrame(top_matches_json)

top_matches_df['similarity_value'] = top_matches_df['similarity'].apply(lambda x: x.split()[1]).astype(float)
top_matches_df['factcheck_index'] = top_matches_df['similarity'].apply(lambda x: x.split()[0])

over_50 = top_matches_df.loc[top_matches_df['similarity_value'] >= .5]

over_50 = over_50.sort_values('similarity_value', ascending=False)

over_50 = over_50[['filename', 'input_statement', 'checked_false_statement', 'similarity_value', 'factcheck_index']]

over_50_csv_filepath = Path().cwd().parent.joinpath('data/output_csv/potential_misinformation_with_context.csv')

print("Generating csv file...")

over_50.to_csv(f"{over_50_csv_filepath}")


print("Adding translation...")

lt = LibreTranslateAPI("http://localhost:5000")

def libretranslate_spanish(input_text):
    return lt.translate(f"{input_text}", "es", "en")

def libretranslate_french(input_text):
    return lt.translate(f"{input_text}", "fr", "en")

try:
    over_50['translation'] = over_50['input_statement'].apply(lambda x: libretranslate_french(x))
    over_50['translation'] = over_50['translation'].apply(lambda x: libretranslate_spanish(x))

    over_50 = over_50.sort_values('similarity_value', ascending=False)

    over_50 = over_50[['filename', 'input_statement', 'translation', 'checked_false_statement', 'similarity_value', 'factcheck_index']]

    print("Generating csv file...")

    translation_filepath = Path().cwd().parent.joinpath('data/output_csv/potential_misinformation_with_translations.csv')

    over_50.to_csv(f"{translation_filepath}")

except:
    print("Failed to add transcription. Check that LibreTranslate is running.")

Starting search of all transcripts...
Error checking statement 1
Error checking statement 2
Error checking statement 3
Error checking statement 4
Error checking statement 5
Error checking statement 6
Error checking statement 7
Error checking statement 8
Error checking statement 9
Error checking statement 10
Error checking statement 11
Error checking statement 12
Error checking statement 13
Error checking statement 14
Error checking statement 15
Error checking statement 16
Error checking statement 17
Error checking statement 18
Error checking statement 19
Error checking statement 20
Error checking statement 21
Error checking statement 22
Error checking statement 23
Error checking statement 24
Error checking statement 25
Error checking statement 26
Error checking statement 27
Error checking statement 28
Error checking statement 29
Error checking statement 30
Error checking statement 31
Error checking statement 32
Error checking statement 33
Error checking statement 34
Error checking stat

KeyError: 'similarity'